In [45]:
#!/usr/bin/env python3
#import your libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.metrics import recall_score
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import binarize
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [46]:
train_transaction = pd.read_csv('/Users/krahman/work/fraud_detection/data/train_transaction.csv')
train_identity = pd.read_csv('/Users/krahman/work/fraud_detection/data/train_identity.csv')
# merging dataframes 
df_train = train_transaction.merge(train_identity, on='TransactionID', how='left')

In [47]:
# df_train

In [48]:
# train_transaction.head()

In [49]:
# train_identity.head()

In [50]:
# train_transaction.info();

In [51]:
# train_identity.info();

In [52]:
# df_train.shape

In [53]:
# train_transaction_sum = train_transaction.duplicated().sum()
# train_identity_sum = train_identity.duplicated().sum()
# columns = train_transaction.columns
# print('Train transaction duplicates: {}\nTrain identity duplicates: {} \n'.format(train_transaction_sum, train_identity_sum))
# # print('Train feature columns:\n', list(columns))

### Assigning columns to specific lists (cat, num, date, etc.)

In [54]:
# dropping columns with more than 50% missing data
length_df = df_train.shape[0]/2
list_temp = []
for val in df_train.columns:
    if np.sum(df_train[val].isnull()) > length_df:
        list_temp.append(val)   
df_train = df_train.drop(list_temp, axis=1)

###################################
# c is num, ex, how many addresses associated with card
col_c = [c for c in df_train.columns if c.startswith('C') and (len(c)==2 or len(c)==3)]
# d is num, time/days between transactions
col_d = [d for d in df_train.columns if d.startswith('D') and (len(d)==2 or len(d)==3)]
# m is date of transaction
col_m = [m for m in df_train.columns if m.startswith('M') and (len(m)==2 or len(m)==3)]
# v is num, features created by vesta such as ranking, counting. entity relationships, etc. 
col_v = [v for v in df_train.columns if v.startswith('V') and (len(v)==2 or len(v)==3 or len(v)==4)]
# i is identity information like network and digital signature associated with transaction
col_i = [i for i in df_train.columns if i.startswith('id_') and len(i)==5]
# ca is cat, card information such as card type, etc. 
col_card = [ca for ca in df_train.columns if ca.startswith('card')]
# D = time elapsed between each transaction, card = card information, C = counting, ie how many addresses 
# associated with card, M=True/False, V created features on ranking, counting, etc. 

# column id and target
col_id = ['TransactionID']
col_target = ['isFraud']

# converting categorical columns with numerical values to string types.
col_cat_to_obj = ['addr1','addr2','card1','card2', 'card3', 'card5']
for val in col_cat_to_obj:
    df_train[val] = df_train[val].astype(str)

# categorical columns
col_cat = ['addr1','addr2','ProductCD',"P_emaildomain"] + col_card + col_m

# C counter, D is time elapsed between transactions, V feature engineered variables by firm
col_num = ['TransactionAmt'] + col_c + col_d + col_v

# figure out how to handle this. What do these dates mean? Do certain dates have more fraud occurences?
col_date = ['TransactionDT'] 

# boolean columns. convert via dummy variable. We dont know if true/false is better than one or the other. 
# col_bool = col_m

# confirming all columns are accounted for
print('Total columns: ' + str(len(col_cat + col_num + col_date + col_id + col_i + col_target)))

# col_all = col_cat + col_num + col_date + col_bool + col_id + col_target
# columns removed dist1, dist2, R_emaildomain, DeviceInfo, DeviceType 

Total columns: 220


In [55]:
class Preprocessing:
    def __init__(self):
        '''initialize variables and column names for null dataframe'''
        self.df_train = df_train
        self.list_col = []
        self.list_total = []
        self.dict_unique = {}
        self.list_datatype = []
        self.list_unique_val = []
        self.list_mode_count = []
        self.list_mode_value = []
        self.list_mode_count_perc = []
        self.list_unique_total = []
        self.list_unique_first_10 = []
        self.column_names = ['col_name', 'total_null', 'datatype', 'total_unique',
                             'mode_value', 'mode_count', 'mode_percentage']

    def missing_values(self):
        '''check for null values and add to null dataframe if more than 0 nulls exist'''
        for val in df_train.columns:
            total_null = np.sum(df_train[val].isnull())
            if total_null > 0:
                self.list_col.append(val)
                self.list_total.append(total_null)
                self.list_datatype.append(df_train[val].dtype)
                self.list_unique_total.append(len(df_train[val].unique()))
                self.list_unique_val.append(df_train[val].unique())
                self.list_mode_value.append(df_train[val].mode()[0])
                val_counts = max(df_train[val].value_counts())
                self.list_mode_count.append(val_counts)
                self.list_mode_count_perc.append(val_counts/len(df_train))
                val_unique = df_train[val].unique()
                self._create_dict(val_unique, df_train, val)
        df_null_info = self._create_dataframe()
        df_null_info = self._create_df_unique(df_null_info)
        self._summary(df_null_info)
        self._fillna(df_null_info)
        return df_null_info
    
    def _create_dict(self, val_unique, df_train, val):
        '''create dictionary of unique values for each column'''
        if (len(val_unique) > 99) and isinstance(df_train[val], object):  
            self.dict_unique.update([(val,0)])
        if (len(val_unique) > 99) and not isinstance(df_train[val], object):
            self.dict_unique.update([(val,0)])
        if len(val_unique) < 100:
            self.dict_unique.update([(val, val_unique)])

    def _create_dataframe(self):
        '''create main dataframe'''
        df_null_info = pd.DataFrame()
        counter = -1
        for list_val in [self.list_col, self.list_total, self.list_datatype, self.list_unique_total,
                        self.list_mode_value, self.list_mode_count, self.list_mode_count_perc]:
            counter = counter + 1
            col_title = self.column_names[counter]
            df = pd.DataFrame(list_val, columns=[col_title])
            df_null_info = pd.concat([df_null_info, df], axis=1)
        return df_null_info
    
    def _summary(self, df_null_info):
        val = df_null_info.shape[0]
        print('There were ' + str(val) + ' columns with null values.')
    
    def _create_df_unique(self, df_null_info):
        '''create unique values dataframe'''
        series_unique = pd.Series(self.dict_unique)
        df_unique = pd.DataFrame(series_unique).reset_index()
        df_unique = df_unique.rename(columns={'index':'col_name', 0:'unique'})
        df_null_info = df_null_info.merge(df_unique, how='left', 
                                          left_on='col_name', right_on='col_name')
        df_null_info.to_csv('/Users/krahman/work/fraud_detection/saved_files/df_null_info.csv')
        return df_null_info
    
    def _fillna(self, df_null_info):
        '''fill null values of df_train with mode'''
        total_null_columns = sum(np.sum(self.df_train.isnull()))
        if total_null_columns > 0:
            for val in df_null_info.col_name:
                val_mode = self.df_train[val].mode()[0]
                self.df_train[val] = self.df_train[val].fillna(val_mode)
                
    def impute_features(self):
        df_temp = pp.df_train
        for val in col_cat:
            total_unique_val = pp.df_train[val].unique().shape[0]
            if len(df_temp[val].unique()) < 10:
                print('dummies encoded: ' + str(val) + ' unique ' + str(total_unique_val))
                df_dumm = pd.get_dummies(df_temp[val], prefix=val, drop_first=True)
                df_temp = df_temp.drop(val,axis=1)
                df_temp = pd.concat([df_temp, df_dumm], axis=1)
            else:
                le = LabelEncoder()
                df_temp[val] = le.fit_transform(df_temp[val])
                print('label encoded: ' + str(val) + ' unique ' + str(total_unique_val))
        print('new dataframe shape:' + str(df_temp.shape))
        return df_temp

pp = Preprocessing()
df_null_info = pp.missing_values()
# df_null_info

# determine what to do with columns that have too many unique values... obviously.. types of solutions
# would be to put "MISSING" for those that dont have an email address... but you will need to evaluate 
# and make instead a counter of unique values, then append that and look at the CSV via google sheets. use 
# something like the code below 

# Planning - our preprocessing method must automatically drop missing values, but we can't do that because
# we need to see about filling them in first, then decide if we need to drop them. Right now, we need to
# create a dataframe that shows unique values for each column with missing values. 

# we need to look at each variable and see if it's unique or categorical. We need to use possibly PCA...? How do
# we handle so many variables? 

There were 195 columns with null values.


In [56]:
pp.df_train = pp.impute_features()

label encoded: addr1 unique 333
label encoded: addr2 unique 75
dummies encoded: ProductCD unique 5
label encoded: P_emaildomain unique 59
label encoded: card1 unique 13553
label encoded: card2 unique 501
label encoded: card3 unique 115
dummies encoded: card4 unique 4
label encoded: card5 unique 120
dummies encoded: card6 unique 4
dummies encoded: M1 unique 2
dummies encoded: M2 unique 2
dummies encoded: M3 unique 2
dummies encoded: M4 unique 3
dummies encoded: M6 unique 2
new dataframe shape:(590540, 228)


In [ ]:
# # c is num, ex, how many addresses associated with card
# col_c = [c for c in df_train.columns if c.startswith('C') and (len(c)==2 or len(c)==3)]
# # d is num, time/days between transactions
# col_d = [d for d in df_train.columns if d.startswith('D') and (len(d)==2 or len(d)==3)]
# # m is date of transaction
# col_m = [m for m in df_train.columns if m.startswith('M') and (len(m)==2 or len(m)==3)]
# # v is num, features created by vesta such as ranking, counting. entity relationships, etc. 
# col_v = [v for v in df_train.columns if v.startswith('V') and (len(v)==2 or len(v)==3 or len(v)==4)]
# # i is identity information like network and digital signature associated with transaction
# col_i = [i for i in df_train.columns if i.startswith('id_') and len(i)==5]
# # ca is cat, card information such as card type, etc. 
# col_card = [ca for ca in df_train.columns if ca.startswith('card')]

In [ ]:
# # c is num, ex, how many addresses associated with card
# col_c = [c for c in df_train.columns if c.startswith('C') and (len(c)==2 or len(c)==3)]
# # d is num, time/days between transactions
# col_d = [d for d in df_train.columns if d.startswith('D') and (len(d)==2 or len(d)==3)]
# # m is date of transaction
# col_m = [m for m in df_train.columns if m.startswith('M') and (len(m)==2 or len(m)==3)]
# # v is num, features created by vesta such as ranking, counting. entity relationships, etc. 
# col_v = [v for v in df_train.columns if v.startswith('V') and (len(v)==2 or len(v)==3 or len(v)==4)]
# # i is identity information like network and digital signature associated with transaction
# col_i = [i for i in df_train.columns if i.startswith('id_') and len(i)==5]
# # ca is cat, card information such as card type, etc. 
# col_card = [ca for ca in df_train.columns if ca.startswith('card')]
# # D = time elapsed between each transaction, card = card information, C = counting, ie how many addresses 
# # associated with card, M=True/False, V created features on ranking, counting, etc. 

# # column id and target
# col_id = ['TransactionID']
# col_target = ['isFraud']

# # converting categorical columns with numerical values to string types.
# col_cat_to_obj = ['addr1','addr2','card1','card2', 'card3', 'card5']
# for val in col_cat_to_obj:
#     df_train[val] = df_train[val].astype(str)

# # categorical columns
# col_cat = ['addr1','addr2','ProductCD',"P_emaildomain"] + col_card + col_m

# # C counter, D is time elapsed between transactions, V feature engineered variables by firm
# col_num = ['TransactionAmt'] + col_c + col_d + col_v

# # figure out how to handle this. What do these dates mean? Do certain dates have more fraud occurences?
# col_date = ['TransactionDT'] 

# # boolean columns. convert via dummy variable. We dont know if true/false is better than one or the other. 
# # col_bool = col_m

# # confirming all columns are accounted for
# print('Total columns: ' + str(len(col_cat + col_num + col_date + col_id + col_i + col_target)))

# # col_all = col_cat + col_num + col_date + col_bool + col_id + col_target
# # columns removed dist1, dist2, R_emaildomain, DeviceInfo, DeviceType 

In [ ]:
# plt.scatter(X.card1, y)
# sns.regplot(x='card5_237.0', y='isFraud', data=pp.df_train, logistic=True, color='b')

## Splitting features dataframe

In [57]:
# create features, target and split the dataframe
X = pp.df_train.drop(col_target, axis=1)
X = X.drop(col_id, axis=1)
y = pp.df_train[col_target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1, random_state=42)

### Test Dropping Features On Base Model

In [9]:
###########
# label encoded: addr1 unique 333
# dummies encoded: addr2 unique 75
# dummies encoded: ProductCD unique 5
# dummies encoded: P_emaildomain unique 59
# label encoded: card1 unique 13553
# label encoded: card2 unique 501
# label encoded: card3 unique 115
# dummies encoded: card4 unique 4
# label encoded: card5 unique 120
# dummies encoded: card6 unique 4
# dummies encoded: M1 unique 2
# dummies encoded: M2 unique 2
# dummies encoded: M3 unique 2
# dummies encoded: M4 unique 3
# dummies encoded: M6 unique 2

In [59]:
# Testing dropping columns
model_lr_results = pd.DataFrame()
model_recall = []
model_precision = []
model_auc_score = []

for col in ['addr1', 'addr2', 'P_emaildomain', 'card1', 'card2', 'card3', 'card5']:
    X_drop = X.drop(col, axis=1)
    X_drop = X_drop.loc[:1000,:]
    y_drop = y[:1001]
    
    scaled_X = StandardScaler().fit_transform(X_drop)
    # pca
    pca = PCA()
    pcomponents = pca.fit_transform(scaled_X)
    X_pca = pd.DataFrame(data=pcomponents)
    # split
    X_train2, X_test2, y_train2, y_test2 = train_test_split(X_pca, y_drop, test_size=0.1, random_state=42)
    # smote
    sm = SMOTE(random_state=42, ratio=1.0, n_jobs=-1)
    X_train_res, y_train_res = sm.fit_sample(X_train2, y_train2)
    # model fit
    model_lr_pca = LogisticRegression(random_state=42)
    model_lr_pca.fit(X_train_res, y_train_res)
    # predict
    y_pred_prob = model_lr_pca.predict_proba(X_pca)
    y_pred_class = binarize(y_pred_prob, 0.5)[:,1]
    # scoring
    model_recall.append(recall_score(y_drop, y_pred_class))
    model_precision.append(precision_score(y_drop, y_pred_class))
    model_auc_score.append(roc_auc_score(y_drop, y_pred_prob[:,1]))
    print(confusion_matrix(y_drop, y_pred_class))
    print(classification_report(y_drop, y_pred_class))
    print('AUC: ', roc_auc_score(y_drop, y_pred_prob[:,1]))

model_recall = pd.Series(model_recall, name='recall')
model_precision = pd.Series(model_precision, name='precision')
model_auc_score = pd.Series(model_auc_score, name='auc_score')
model_results_final = pd.concat([model_auc_score, model_recall, model_precision],axis=1)
model_results_final

[[960  26]
 [  0  15]]
              precision    recall  f1-score   support

           0       1.00      0.97      0.99       986
           1       0.37      1.00      0.54        15

    accuracy                           0.97      1001
   macro avg       0.68      0.99      0.76      1001
weighted avg       0.99      0.97      0.98      1001

AUC:  0.9961460446247464
[[964  22]
 [  0  15]]
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       986
           1       0.41      1.00      0.58        15

    accuracy                           0.98      1001
   macro avg       0.70      0.99      0.78      1001
weighted avg       0.99      0.98      0.98      1001

AUC:  0.9963488843813387
[[963  23]
 [  0  15]]
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       986
           1       0.39      1.00      0.57        15

    accuracy                           0.98      1001
   macro 

,auc_score,recall,precision
0,0.996146,1.0,0.365854
1,0.996349,1.0,0.405405
2,0.996078,1.0,0.394737
3,0.996146,1.0,0.394737
4,0.996687,1.0,0.394737
5,0.996349,1.0,0.405405
6,0.996755,1.0,0.375000


## Applying PCA (2 components)

In [ ]:
# standardizing our data, which is required for PCA.
scaled_X = StandardScaler().fit_transform(X)
pd.DataFrame(scaled_X, columns=X.columns).head()

In [ ]:
# PCA instantiate and fit 
pca = PCA(n_components=2)
pcomponents = pca.fit_transform(scaled_X)
X_pca = pd.DataFrame(data = pcomponents, columns=['PC1','PC2'])
print(X_pca.shape)
X_pca.head()

In [ ]:
# two principal components scatter plot
plt.figure(figsize=(8,6))
plt.scatter(X_pca['PC1'], X_pca['PC2'], c=y['isFraud'])
plt.xlabel('First principal component')
plt.ylabel('Second principal component')

# explaining vaariance
print('Variance ratio:')
print(pca.explained_variance_ratio_)
# interpreting principal components
print('\nPrincipal components explained:')
pd.DataFrame(pca.components_, columns=list(X.columns), index=('PC1', 'PC2'))

## Applying PCA to all features (all components)

In [ ]:
pca2 = PCA().fit(scaled_X)
# pca2.explained_variance_ratio_
# np.cumsum(pca2.explained_variance_ratio_)

In [ ]:
plt.plot(np.cumsum(pca2.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)')
plt.title('Credit Card Fraud Explained Variance')
plt.show()

## Model w/ SMOTE only - base model

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
# Apply SMOTE
sm = SMOTE(random_state=42, ratio=1.0, n_jobs=-1)
X_train_res, y_train_res = sm.fit_sample(X_train1, y_train1)

In [ ]:
model_lr = LogisticRegression(random_state=42)
model_lr.fit(X_train_res, y_train_res) 

In [ ]:
# predicting on test set w/o PCA
print("Predicting using only SMOTE (and w/o PCA)\n")
y_pred_test1 = model_lr.predict(X_test1)
print("Test set:")
print("Validation results")
print(model_lr.score(X_test1, y_test1))
print(recall_score(y_test1, y_pred_test1))
print("\nConfusion Matrix")
print(confusion_matrix(y_test1, y_pred_test1))
print('\nClassification Report:\n')
print(classification_report(y_test1, y_pred_test1))

# predicting on original dataset
print("Whole dataset:")
y_pred = model_lr.predict(X)
print("\nTest Results")
print(model_lr.score(X, y))
print(recall_score(y, y_pred))
print("\nConfusion Matrix")
print(confusion_matrix(y, y_pred))
print('\nClassification Report:\n')
print(classification_report(y, y_pred))

## Logistic Regression w/PCA  w/SMOTE

In [ ]:
X_drop = X.drop('card1', axis=1)

In [ ]:
# applying PCA
scaled_X = StandardScaler().fit_transform(X_drop)
pca = PCA(n_components=250)
pcomponents = pca.fit_transform(scaled_X)
X_pca = pd.DataFrame(data=pcomponents)

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_pca, y, test_size=0.1, random_state=42)

In [ ]:
# applying SMOTE
sm = SMOTE(random_state=42, ratio=1.0, n_jobs=-1)
X_train_res, y_train_res = sm.fit_sample(X_train2, y_train2)

In [ ]:
# fitting logistic regression
model_lr_pca = LogisticRegression(random_state=42)
model_lr_pca.fit(X_train_res, y_train_res)

In [ ]:
# notes, do we need to visualize our results?? We need to look at our confusion matrix
# and decide how to improve our results from here... Test and see what happens if we 
# increase our principal components... test removing columns, adding columns, imputing
# certain columns all together. 

# test imputing 500 or less for ohe. Then test dropping card1 due to its number
# of unique values that would make our data very high dimensional. We can try only ohe for
# the entire dataset to see how our model performs over all... Though.. it will likely run 
# out of memory and crash. 

# ALSO test probabilities on logisticregression


### Predict Proba

In [ ]:
%matplotlib inline
plt.rcParams['font.size'] = 14

In [ ]:
y_pred_prob = model_lr_pca.predict_proba(X_pca)
plt.hist(y_pred_prob[:,1], bins=8)
plt.xlim(0,1)
plt.title("Histogram of Probability of Fraud")
plt.xlabel("Predicted probability of Fraud")
plt.ylabel("Frequency")

In [ ]:
y_pred_prob = model_lr_pca.predict_proba(X_pca)
y_pred_class = binarize(y_pred_prob, 0.5)[:,1]

In [ ]:
print(confusion_matrix(y, y_pred_class))
print(classification_report(y, y_pred_class))

In [ ]:
fpr, tpr, thresholds = roc_curve(y, y_pred_prob[:,1])
plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.title("ROC curve for fraud detection classifier")
plt.xlabel("False Positive Rate (1 - Specificity)")
plt.ylabel("True Positive Rate (Sensitivity)")
plt.grid(True)

In [ ]:
def evaluate_threshold(threshold):
    print("Sensitivity:", tpr[thresholds > threshold][-1])
    print("Specificity:", 1 - fpr[thresholds > threshold][-1],'\n')

In [ ]:
evaluate_threshold(.5)
evaluate_threshold(.2)
evaluate_threshold(.1)

print(roc_auc_score(y, y_pred_prob[:,1]))
# NEXT CONT, from here on youtube vid 49:00. once we finish the video
# go back to improving the model using feature selection, eda, 
# and decision tree eda

In [ ]:
y_pred_actual = model_lr_pca.predict(X_test2) # actual prediction test data set
print('y_pred_actual on test set\n')
print(y_pred_actual[0:10])
print(confusion_matrix(y_test2, y_pred_actual))
print(classification_report(y_test2, y_pred_actual))

print('y_pred_proba\n')
y_pred_proba = model_lr_pca.predict_proba(X_test2)
y_pred_class = binarize(y_pred_proba, 0.5)[:,1]
print(y_pred_class[0:10])
print(confusion_matrix(y_test2, y_pred_class))
print(classification_report(y_test2, y_pred_class))

In [ ]:
y_pred_actual = model_lr_pca.predict(X_pca) # actual full data set
print('Logistic Regression')
print('y_pred_actual full data set\n')
print(y_pred_actual[0:10])
print(confusion_matrix(y, y_pred_actual))
print(classification_report(y, y_pred_actual))

print('y_pred_proba full data set\n')
y_pred_proba = model_lr_pca.predict_proba(X_pca)#[:,1]#[0:10]
y_pred_class = binarize(y_pred_proba, 0.2)[:,1]
print(y_pred_class[0:10])
print(confusion_matrix(y, y_pred_class))
print(classification_report(y, y_pred_class))

In [ ]:
##################

In [ ]:
# predicting on test set w/o PCA
print("Predicting using Logistic Regression, PCA, SMOTE\n")
y_pred_pca_sm = model_lr_pca.predict(X_test2)
# y_pred_test1 = model_lr.predict(X_test1)
print("Test set:")
print("Validation results")
print(model_lr_pca.score(X_test2, y_test2))
print(recall_score(y_test2, y_pred_pca_sm))
print("\nConfusion Matrix")
print(confusion_matrix(y_test2, y_pred_pca_sm))
print('\nClassification Report:\n')
print(classification_report(y_test2, y_pred_pca_sm))

# predicting on original dataset
print("Whole dataset:")
y_pred_pca_sm_whole = model_lr_pca.predict(X_pca)
print("\nTest Results")
print(model_lr_pca.score(X_pca, y))
print(recall_score(y, y_pred_pca_sm_whole))
print("\nConfusion Matrix")
print(confusion_matrix(y, y_pred_pca_sm_whole))
print('\nClassification Report:\n')
print(classification_report(y, y_pred_pca_sm_whole))

### Model - Decision Tree - w/PCA w/SMOTE

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model_dt_pca_smote = DecisionTreeClassifier(random_state=42)
model_dt_pca_smote.fit(X_train_res, y_train_res)

In [ ]:
# predicting on test set w/o PCA
print("Predicting using PCA\n")
y_pred_pca = model_dt_pca_smote.predict(X_test2)
# y_pred_test1 = model_lr.predict(X_test1)
print("Test set:")
print("Validation results")
print(model_dt_pca_smote.score(X_test2, y_test2))
print(recall_score(y_test2, y_pred_pca))
print("\nConfusion Matrix")
print(confusion_matrix(y_test2, y_pred_pca))
print('\nClassification Report:\n')
print(classification_report(y_test2, y_pred_pca))

# predicting on original dataset
print("Whole dataset:")
y_pred_pca = model_dt_pca_smote.predict(X_pca)
print("\nTest Results")
print(model_dt_pca_smote.score(X_pca, y))
print(recall_score(y, y_pred_pca))
print("\nConfusion Matrix")
print(confusion_matrix(y, y_pred_pca))
print('\nClassification Report:\n')
print(classification_report(y, y_pred_pca))

In [ ]:
#####################

In [ ]:
print("Variance ratio:")
print(pca.explained_variance_ratio_)
print("\nPrincipal components explained:")
pd.DataFrame(pca.components_, columns=list(X.columns), index=('PC1', 'PC2'))

In [ ]:
# # explaining variance
# print('Variance ratio:')
# print(pca.explained_variance_ratio_)
# # interpreting principal components
# print('\nPrincipal components explained:')
# pd.DataFrame(pca.components_, columns=list(X.columns), index=('PC1', 'PC2'))

# # # predicting on original dataset
# # y_pred = clf_lr.predict(X)
# # print("\nTest Results")
# # print(clf_lr.score(X, y))
# # print(recall_score(y, y_pred))
# # print("\nConfusion Matrix")
# # print(confusion_matrix(y, y_pred))
# # print('\nClassification Report:\n')
# # print(classification_report(y, y_pred))

In [ ]:
# scaled_X2 = StandardScaler().fit_transform(X)
# pca3 = PCA(n_components=275)
# pcomponents = pca3.fit_transform(scaled_X2)
# X_pca = pd.DataFrame(data=pcomponents)

## Applying SMOTE

In [ ]:
# applying SMOTE to train set to correct class imbalance
sm = SMOTE(random_state=42, ratio = 1.0, n_jobs=-1)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train)

In [ ]:
# fitting to residuals created by SMOTE
clf_lr = LogisticRegression(class_weight='balanced', random_state=42)
clf_lr.fit(X_train_res, y_train_res);

In [ ]:
# predicting on test set
y_test_pred = clf_lr.predict(X_test)
print("Validation results")
print(clf_lr.score(X_test, y_test))
print(recall_score(y_test, y_test_pred))
print("\nConfusion Matrix")
print(confusion_matrix(y_test, y_test_pred))
print('\nClassification Report:\n')
print(classification_report(y_test, y_test_pred))

In [ ]:
# predicting on original dataset
y_pred = clf_lr.predict(X)
print("\nTest Results")
print(clf_lr.score(X, y))
print(recall_score(y, y_pred))
print("\nConfusion Matrix")
print(confusion_matrix(y, y_pred))
print('\nClassification Report:\n')
print(classification_report(y, y_pred))

In [ ]:
# print("\nConfusion Matrix")
# print(confusion_matrix(y, y_pred))
# print('\nClassification Report:\n')
# print(classification_report(y, y_pred))

In [ ]:
# logistic regression
# from sklearn.linear_model import LogisticRegression
# clf = LogisticRegression(class_weight='balanced').fit(X_train, y_train)
# clf.predict(X_test)
# clf.score(X_test, y_test)
# 0.7870216903822372 
# 0.8013998429794899 < 60
# 0.5085411973583608 < 1000
# 0.63556873752431 < 60 dropped col_id

In [ ]:
# clf.predict_proba(X_test)

In [ ]:
# # confusion matrix
# y_pred = clf.predict(X_test)
# print(confusion_matrix(y_test, y_pred))

In [ ]:
# print(classification_report(y_test, y_pred))

In [ ]:
# read up on class im balance and correct it. 
# perhaps we can one by one run our model through a decision tree and do one hot encoding for one big 
# categorical column at a time lets say 13,000 unique values, then we can see of the new 13,000 columns we
# have if any actually have predictive value for us. 

In [ ]:
# import statsmodels.api as sm

# # define iv
# iv = X.columns

# # fit the logistic regression function
# logReg = sm.Logit(y_train, X_train)
# answer = logReg.fit()

In [ ]:
# df_result['Fraud']

In [ ]:
# for val in pp.df_train.columns:
#     if (pp.df_train[pp.df_train[val]=='nan'].shape[0]) > 0:
#         print(val)

In [ ]:
# # pp.df_train
# # lets find out which columns are object... 
# list_col_object = []
# for val in pp.df_train.columns:
#     if pp.df_train[val].dtype=='O':
#         list_col_object.append(val)
        
# pp.df_train[list_col_object]
# # for card2, nan was the most commonly seen value... so it imputed that...? 

In [ ]:
# for val in pp.df_train.columns:
#     if pp.df_train[pp.df_train[val]=='nan']

In [ ]:
# list_col_object = []
# for val in pp.df_train.columns:
#     if pp.df_train[val]:
#         list_col_object.append(val)

In [ ]:
# np.sum(pp.df_train[list_col_object].isnull())
# pp.df_train['card2'].unique()
# pp.df_train[df_train['card2']=='nan']

In [ ]:
# sum(np.sum(pp.df_train.isnull()))

In [ ]:
# We need to clean the data before thinking about applying PCA. 
# 1. Determine which columns are continuous, which are ranking.
# 2. Determine which columns are bool (easy)
# 3. Determine which columns are categorical, then impute with pandas (we dont know which columns means what
#    so we cant assume True or better than False, etc.)
# 4. After 

# 1. impute all objects columns with one hot encoding
# 1.1 What's a categorical? 
# We know the V's are ranking.. we need to discern meaning, 

# 2. figure out if we should do pca next. we should do that next..
# 3. then stand up the model. 


In [ ]:
# plt.bar(df_train['V14'])
# sns.barplot(df_train['V196'])
# we need to imput the mode here.. 
# df_train['V14'].mode()
# df_train['V22'].unique()
# for val in col_v:
#     print(val)
#     print(df_train[val].unique())
# we ned to descern what is a 0 1 outcome then impute.

# col = 'V290'
# series_temp = df_train[col].fillna(df_train[col].mode()[0])
# plt.hist(series_temp);
# missing_val = np.sum(df_train[col].isnull())
# print('Missing values: ' + str(missing_val))
# print("REAL VALUE COUNTS: ")
# df_train[col].value_counts().head()

# col = 'card4'
# series_temp = df_train[col].fillna(df_train[col].mode()[0])
# plt.hist(series_temp);
# df_train[col].value_counts()

# col = 'D1'
# series_temp = df_train[col].fillna(df_train[col].mean())
# plt.hist(series_temp);
# df_train['D1'].value_counts()